# SeqIO

Manipulating nucleotide and protein sequences
- manipulating simple files (e.g., FASTA files) 
- mainpulating files with rich meta data annotation (genbank file format)

Biopython has an API for IO of sequences (__Bio.SeqIO__) and __Bio.SeqRecord__ for accessing richly annoted data

## The SeqIO object
- supports different file formats
- for example: clustal(aln), genbank(gb), pir, embl, fasta, ...
- see [SeqIO wiki](http://biopython.org/wiki/SeqIO) for more information


SeqIO provides four functions: 
- __parse()__: iteratively parse all elements in the file
- __read()__: parse a one-element file and return the element
- __write()__: write elements to a file
- __convert()__: parse one format and immediately write another

SeqIO manipulates sequences as __Seqrecord__ objects
Alignments (Bio.AlignIo), Blast results (Bio.Blast), and phylogenetic trees(Bio.Phylo) use the same input/output conventions

## The SeqRecord object
SeqRecord is a common object for sequence file entries
- wraps a __Seq__ object
- attaches _metadata_ (like identifiers and annotation)
- Suitable for richly annotated sequence (like data from gb and embl files)


### Attributes of a SeqRecord object:
- __seq__ the sequence itself, typically (Bio.Seq) object
- __id__ primary ID for the sequence (string), typically an accession number
- __name__ "common" name for the sequence (LOCUS id in gb files)
- __description__ human-readible description of the sequence
- __letter_annotations__ dictionary of additional info about individual letters in the sequence (like quality scores, secondary structure information)
- __annotations__ dictionary of additional unstructured info
- __features__ list of __SeqFeature__ objects with more structured information (like position of genes on a genome and domains on  a protein sequence)
- __dbxrefs__ list of database cross-references(strings)

### Creation of SeqRecord objects:
#### 1.) From scratch: directly setting the attributes:


In [ ]:
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
simple_seq = Seq("GATC")
simple_seq_r = SeqRecord(simple_seq)
simple_seq_r.id = "AC12345" # id is important for outputting the record to a file
print(simple_seq_r) # all other attributes are empty or unknown

# set annotations (is a python dictionary)
simple_seq_r.annotations["evidence"] = "None. I just made it up."
print(simple_seq_r.annotations)



#### 2.) From a sequence file 
### Bio.SeqIO.parse():
1. Open the sequence file to obtain a handle
2. Pass the handle to SeqIO.parse():
   Arguments:
     - file handle or file name
     - file format (explicit specification)
     - alphabet (optional)
   Output: SeqRecord iterator
3. Close the sequence file 
4. Display the content of the sequence records (via the iterator)

__Note__: In python it is good practice to open files with the "with" statement to create a context manager. This has the advantage, that the file is properly closed (even if an exception was thrown). See [python's documentation on with](https://docs.python.org/3.5/reference/datamodel.html#context-managers) for more information.

In [ ]:
from Bio import SeqIO
with open("data/1ATP_ncbi.fasta", "r") as in_file:
    seq_rec_it = SeqIO.parse(in_file, "fasta")   
    
    # iterate through the records 
    for rec in seq_rec_it:
        print(rec.id, len(rec.seq), rec.seq[:10] + "...")
        
   
    

In [ ]:
# or generate a list of record iterators
with open("data/1ATP_ncbi.fasta", "r") as in_file:
   
    recs = list(SeqIO.parse(in_file, "fasta"))
    # and print only the first record
    print(recs[0].id, len(recs[0].seq), recs[0].seq[:10] + "...") 

In [ ]:
# or for small files, generate a dictionary from the input file
with open("data/1ATP_ncbi.fasta", "r") as in_file:
    rec_dict = SeqIO.to_dict(SeqIO.parse(in_file, "fasta"))
    # print the keys of the dictionary
    print(rec_dict.keys())


### Bio.SeqIO.read():
- Same arguments as Bio.SeqIO.parse()
- use it when you know, that your file only contains one record!
- returns a single Bio.SeqRecord object

Example: Parsing a genbank file with rich text format

In [ ]:
from Bio import SeqIO

# using SeqIO's parse method to get the length of the sequence of the record
print(sum([len(r) for r in SeqIO.parse("data/single_record.gbk", "genbank")]))

# using SeqIO's read nethod to get the length of the sequence 
record = SeqIO.read("data/single_record.gbk", "genbank")
print(len(record.seq))

print(record)

## Bio.SeqIO API: writing into a file
### Bio.SeqIO.write():
- Arguments: SeqRecord-s, file handle (file name), file format
- Output: number of written records

In [ ]:
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
seq_r = SeqRecord(Seq("GATC"))
print(SeqIO.write(seq_r, "data/output/gatc.fasta", "fasta"))

### Bio.SeqIO.convert():
- Arguments: file name, converted file name, file format
- Output: number of converted records


In [ ]:
count = SeqIO.convert("data/single_record.gbk", "genbank", "data/output/conversion.fasta", "fasta")
print("Number of converted records: ", count)

## Bio.SeqIO API: Analysing sequence files

Statistics:

In [ ]:
sizes = [len(r) for r in SeqIO.parse("data/example1.fasta", "fasta")]
print(len(sizes), min(sizes), max(sizes))
print(sizes)

### Example: Plotting a histogram of sequence lengths: 

In [ ]:
from matplotlib import pyplot as plt
plt.hist(sizes, bins = 10)
plt.title("{0} sequences \n sequence lengths {1} to {2}".format(len(sizes), min(sizes), max(sizes)))
plt.xlabel("Sequence length")
plt.ylabel("Count")
plt.show()


### Plotting GC content over all sequences:

In [ ]:
from Bio.SeqUtils import GC
val = sorted(GC(r.seq) for r in SeqIO.parse("data/example1.fasta", "fasta"))
plt.plot(val)
plt.title("{0} sequences \n GC content {1} to {2}".format(len(val), min(val), max(val)))
plt.xlabel("Genes")
plt.ylabel("% GC content")
plt.show()

# AlignIO

## Alignment
- Set of sequences with the __same length __ and __alphabet__
- Different formats, e.g., clustal, stockholm and fasta

## Bio.AlignIO
- Multiple sequence alignment input / output interface
- One/ more sequence alignments represented as alignment objects
- Bio.SeqIO names for supported file formats and API

In [ ]:
from Bio import AlignIO
aln = AlignIO.read("data/msa2.aln", "clustal")
print(aln)

In [ ]:
aln

### Alignment input
- Functions: read(), parse()
- Arguments: file handle, file format, seq_count, alphabet
- Output: (iterator to) MultipleSeqAlignment object(s)

In [ ]:
aln1 = AlignIO.read("data/msa_pk_seed.sth", "stockholm")
print("Alignment length: ", aln1.get_alignment_length())

In [ ]:
for raln in aln1:
    print(raln.seq[:5] + "..." + raln.seq[50:], "\t",  raln.id)

In [ ]:
# parsing a stockholm (PFAM) format (containing rich annotation)
aln2 = AlignIO.read("data/msa_pk_rp15.sth", "stockholm")
print("Alignment length: ", aln2.get_alignment_length())

In [ ]:
for i, raln in enumerate(aln2):
    tmp =""
    if raln.dbxrefs:
        tmp = raln.dbxrefs[0]
    if i < 9:
        print(raln.seq[:5] + "..." + raln.seq[76:],"\t", raln.id, "\t", tmp)

### Alignment output
- Formatted output to a file 
- Functions: write()
- Arguments: alignment-s, file handle, file format




In [ ]:
from Bio.Alphabet import generic_dna
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Align import MultipleSeqAlignment
from Bio import AlignIO

# create the alignment objects from scratch
al1 = MultipleSeqAlignment([
        SeqRecord(Seq("ACTGCTAGCTAG", generic_dna), id = "id1"),
        SeqRecord(Seq("ACT-CTAGCTAG", generic_dna), id = "id2"),])

al2 = MultipleSeqAlignment([
        SeqRecord(Seq("GTCAGC-AG", generic_dna), id = "id3"),
        SeqRecord(Seq("GACAGCTAG", generic_dna), id = "id4")])

# make a list wut tge alignment objects
alignments = [al1, al2]
print("Number of written Alignments: ", AlignIO.write(alignments, "data/output/alignments.fasta", "fasta"))

In [ ]:
# get the alignment in clustal format
print(aln1.format("clustal"))

In [ ]:
# File format conversions
count = AlignIO.convert("data/msa_pk_seed.sth", "stockholm", "data/output/msa_pk_seed.aln", "clustal")
print("Converted alignments: ", count)

# or via AlignIO.read() and AlignIO.write() methods:
alnc = AlignIO.parse("data/msa_pk_seed.sth", "stockholm")
count2 = AlignIO.write(alnc, "data/output/ms_pk_seed.aln", "clustal")

### Manipulating alignments

In [ ]:
alnc = AlignIO.read("data/msa_pk_seed.sth", "stockholm")
print(alnc)
print("___________________________________________________________________________")

# get the number of rows
print(len(alnc))
print("___________________________________________________________________________")

# Access a single sequence letter: Numpy indexing
print(alnc[2,7]) # or pythonic alnc[2].seq[6]
print("___________________________________________________________________________")

# get a single column as a string
print(alnc[:,7])
print("___________________________________________________________________________")

# get a single row as a string and print it with its id
print(alnc[2].seq, alnc[2].id)
print("___________________________________________________________________________")

# remove a block of columns 
print(alnc[:3,:5] + alnc[:3,70:])

## Exercise 4:
Assume you have a working environment defined as follows:
2 	 334878378 	 E 	 350 	 GNA...TEF

In [ ]:
from Bio.Alphabet import IUPAC
from Bio.Seq import Seq
protein = Seq("ERNAK", IUPAC.protein)
dna = Seq("CCGT", IUPAC.unambiguous_dna)

What is the result of the following commands? Can you explain why? 

In [ ]:
protein + dna
protein.complement()

Can a MutableSeq object be a key in a dictionary data structure?

## Exercise 5
Write a function that for the 1ATP_ncbi.fasta file outputs the following:

In [ ]:
File:  data/1ATP_ncbi.fasta
Records:  2
-----------------------------------------------------
Record    ID    Chain    Length    Sequence
1    349840      I     20    TTY...IHD
2    334878378   E     350   GNA...TEF

### Exercise 6

Write a function that filters all records from the in_fasta file satisfying:

1.) length of sequences is higher that 600

2.) the sequence doesn't have unknown nucleotides ("N")

The result is printed in the out_fasta file. The arguments represent the file_names. 
Test the function on the example2.fasta file. 

